# Degridder experiment

Here, we're comparing with the Preesm implementation available at https://gitlab.insa-rennes.fr/Anaelle.Cloarec/degridder

First, let's run and measure all scenarios.

In [1]:
experiment_dir = !realpath ~/repos/iara/experiment/degridder
experiment_dir = experiment_dir[0]

# take only PXX_ scenarios, sort by number

all_scenarios = !ls ~/repos/degridder/Scenarios
all_scenarios = list(filter(lambda x: str(x).startswith("P"), all_scenarios))
all_scenarios.sort(key=lambda x: int(str(x).split('_')[0].strip('P')))
all_scenarios

['P1_bis_medium_1_8_1.scenario',
 'P1_small_1_8_1_complete.scenario',
 'P1_small_1_8_1.scenario',
 'P2_small_32_8_64.scenario',
 'P3_small_32_8_128.scenario',
 'P4_small_64_8_128.scenario',
 'P5_small_64_8_256.scenario',
 'P6_large_1_4_1_complete.scenario',
 'P6_large_1_4_1.scenario',
 'P7_large_32_4_64.scenario',
 'P8_large_32_4_128.scenario',
 'P9_large_64_4_128.scenario',
 'P10_large_64_4_256.scenario',
 'P11_large_1_8_1_complete.scenario',
 'P11_large_1_8_1.scenario',
 'P12_large_32_8_64.scenario',
 'P13_large_32_8_128.scenario',
 'P14_large_64_8_128.scenario',
 'P15_large_64_8_256.scenario',
 'P16_large_64_8_1024.scenario',
 'P17_large_64_4_512.scenario',
 'P18_small_64_8_512.scenario']

In [2]:
# Let's test with just a small one for now.

all_scenarios = ["P1_small_1_8_1.scenario"]

In [3]:


%cd {experiment_dir}
!rm -rf instances
!mkdir -p instances


for scenario in all_scenarios:
  !mkdir -p "{experiment_dir}/instances/{scenario}"
  !mkdir -p "{experiment_dir}/instances/{scenario}/build"
  !rm -rf "{experiment_dir}/instances/{scenario}/build/*"
  ! \time -v -o "{experiment_dir}/instances/{scenario}/time.txt" timeout 5m ~/repos/preesm-cli/commandLinePreesm.sh ~/Downloads/preesm-3.21.0.202501251928-linux.gtk.x86_64/ ~/repos/degridder/ Codegen.workflow {scenario}
  %cd ~/repos/degridder/Code
  ! cmake -B build
  %cd build
  ! make
  !ls
  !pwd
  ! cp degridder_pipeline  {experiment_dir}/instances/{scenario}/build/degridder_pipeline_{scenario.removesuffix('.scenario')}


/home/jabcross/repos/iara/experiment/degridder

***START*** Thu, 31 Jul 2025 10:13:02 -0300


Init workspace and import project

CompileCommand: exclude org/eclipse/jdt/internal/core/dom/rewrite/ASTRewriteAnalyzer.getExtendedRange bool exclude = true
Create.
Opening 'degridder'.

Run workflow from project degridder

CompileCommand: exclude org/eclipse/jdt/internal/core/dom/rewrite/ASTRewriteAnalyzer.getExtendedRange bool exclude = true
10:13:05 NOTICE: Starting workflow execution: degridder/Workflows/Codegen.workflow.10:13:05 NOTICE: Workflow Step: 'PiMM2SrDAG' (id 'pisdf-srdag'): [PiSDFToSingleRateTask] Transforming PiGraph to Single-Rate Directed Acyclic PiGraph.10:13:05 NOTICE: Computing Repetition Vector for graph [top_degridder]10:13:05 NOTICE: top_degridder/config_struct_set_up x1 (total: x1)
10:13:05 NOTICE: top_degridder/broadcast_config x1 (total: x1)
10:13:05 NOTICE: top_degridder/degridder_parallel x1 (total: x1)
10:13:05 NOTICE: top_degridder/fft_shift_complex_to_complex_ac

In [4]:
# get binary sizes

import os

binaries = !ls {experiment_dir}/instances/*/build/*

list(zip(binaries,map(os.path.getsize, binaries)))

[('/home/jabcross/repos/iara/experiment/degridder/instances/P1_small_1_8_1.scenario/build/degridder_pipeline_P1_small_1_8_1',
  1115544)]

Now, let's run our version.

In [7]:
all_scenarios

parameters = {}

# These refer to the input file, not the Preesm scenarios.
scenario_numbers = {
  "small": 1,
  "medium": 2,
  "large": 3
}

scenarios : list = []

%cd {experiment_dir}

# Generate topology.mlir

def instantiateTemplate(_original, _instance, build_dir, size, cores, num_kernel_support, num_chunks):
  GRID_SIZE = 5120 if size == "large" else 2560
  NUM_KERNEL_SUPPORT = int(num_kernel_support)
  OVERSAMPLING_FACTOR = 16
  NUMBER_SAMPLE_IN_KERNEL = ((NUM_KERNEL_SUPPORT + 1) * OVERSAMPLING_FACTOR) ** 2
  NUM_KERNELS = 17
  NUM_SCENARIO = scenario_numbers[size]
  NUM_VISIBILITIES = 7848960 if size == "large" else 3924480
  TOTAL_KERNELS_SAMPLES = NUM_KERNELS * NUMBER_SAMPLE_IN_KERNEL
  NUM_CHUNK = int(num_chunks)
  NUM_VISIB_D_N_CHUNK = NUM_VISIBILITIES // NUM_CHUNK
  assert(NUM_VISIB_D_N_CHUNK * NUM_CHUNK == NUM_VISIBILITIES)
  original = _original
  instance = _instance
  ! rm -f {instance}
  ! cat {original} \
    | sed 's/GRID_SIZE/{GRID_SIZE}/g' \
    | sed 's/NUM_KERNEL_SUPPORT/{NUM_KERNEL_SUPPORT}/g' \
    | sed 's/OVERSAMPLING_FACTOR/{OVERSAMPLING_FACTOR}/g' \
    | sed 's/NUMBER_SAMPLE_IN_KERNEL/{NUMBER_SAMPLE_IN_KERNEL}/g' \
    | sed 's/NUM_KERNELS/{NUM_KERNELS}/g' \
    | sed 's/NUM_SCENARIO/{NUM_SCENARIO}/g' \
    | sed 's/NUM_VISIBILITIES/{NUM_VISIBILITIES}/g' \
    | sed 's/TOTAL_KERNELS_SAMPLES/{TOTAL_KERNELS_SAMPLES}/g' \
    | sed 's/NUM_CHUNK/{NUM_CHUNK}/g' \
    | sed 's/NUM_VISIB_D_N_CHUNK/{NUM_VISIB_D_N_CHUNK}/g' \
    > {instance}

for scenario in all_scenarios:
  size, cores, num_kernel_support, num_chunks = scenario.removesuffix(".scenario").removesuffix("_complete").split("_")[-4:]
  print(size,
  cores,
  num_kernel_support,
  num_chunks)
  topology_file = f"{experiment_dir}/instances/{scenario}/build/topology.mlir"
  main_file = f"{experiment_dir}/instances/{scenario}/build/main.cpp"
  build_dir = f"{experiment_dir}/instances/{scenario}/build"
  instantiateTemplate("topology.mlir.template", topology_file, build_dir, size, cores, num_kernel_support, num_chunks)
  instantiateTemplate("main.cpp.template", main_file, build_dir, size, cores, num_kernel_support, num_chunks)
  scenarios.append({
    "name": scenario,
    "topology_file": topology_file,
    "main_file": main_file,
    "build_dir": build_dir})

scenarios


/home/jabcross/repos/iara/experiment/degridder
small 1 8 1


[{'name': 'P1_small_1_8_1.scenario',
  'topology_file': '/home/jabcross/repos/iara/experiment/degridder/instances/P1_small_1_8_1.scenario/build/topology.mlir',
  'main_file': '/home/jabcross/repos/iara/experiment/degridder/instances/P1_small_1_8_1.scenario/build/main.cpp',
  'build_dir': '/home/jabcross/repos/iara/experiment/degridder/instances/P1_small_1_8_1.scenario/build'}]

In [8]:
# Compile with IaRa.

from timeit import default_timer as timer

source_dir = "~/repos/degridder/Code"

for scenario in scenarios:
  name = scenario["name"]
  topology_file = scenario["topology_file"]
  build_dir = scenario["build_dir"]

  main_actor_name = "top_parallel_degridder_complete" if "complete" in name else "top_parallel_degridder"

  start = timer()

  ! pwd

  %cd {build_dir}

  ! iara-opt --iara-canonicalize --flatten --virtual-fifo='main-actor={main_actor_name} ' "topology.mlir" > "schedule.mlir"

  end = timer()

  scenario["iara_opt_time"] = end - start

  print("Generating schedule took ", end - start)


# CPP_COMPILER=$LLVM_INSTALL/bin/clang++
# C_COMPILER=$LLVM_INSTALL/bin/clang
# COMPILER_FLAGS="-stdlib=libc++ -fopenmp"
# LINKER_FLAGS=" "


  ! sh -x mlir-to-llvmir.sh schedule.mlir

  ! $LLVM_INSTALL/bin/clang -g -fopenmp -xir -c schedule.ll -o schedule.o

  ! $LLVM_INSTALL/bin/clang++ -std=c++26 -g -fopenmp -c -I{source_dir}/include main.cpp -o broadcasts.o

  ! echo {source_dir}


  link_command = ("$LLVM_INSTALL/bin/clang -g -fuse-ld=mold -stdlib=libc++ "
  + "-L$LLVM_INSTALL/lib -lomp -lpthread -lc++ -lc++abi -L/usr/lib64 "
  + "*.o "
  + f" {source_dir}/build/CMakeFiles/degridder_pipeline.dir/src/fft_run.c.o"
  + f" {source_dir}/build/CMakeFiles/degridder_pipeline.dir/src/degridding.c.o"
  + f" {source_dir}/build/CMakeFiles/degridder_pipeline.dir/src/common.c.o"
  + f" {source_dir}/build/CMakeFiles/degridder_pipeline.dir/src/top.c.o"
  + f" {source_dir}/build/CMakeFiles/degridder_pipeline.dir/src/vis_to_csv.c.o"
  # + f"*.o"
  )
  
  ! {link_command}



/home/jabcross/repos/iara/experiment/degridder
/home/jabcross/repos/iara/experiment/degridder/instances/P1_small_1_8_1.scenario/build
iara.node @export_kernels_to_csv
  in
    <<UNKNOWN SSA VALUE>> : tensor<352512x!llvm.struct<"struct.float2", (f32, f32)>>
%0 = iara.node @degridder_actor
  in
    <<UNKNOWN SSA VALUE>> : tensor<352512x!llvm.struct<"struct.float2", (f32, f32)>>
    <<UNKNOWN SSA VALUE>> : tensor<17x!llvm.struct<"struct.int2", (i32, i32)>>
    <<UNKNOWN SSA VALUE>> : tensor<2560x2560x!llvm.struct<"struct.float2", (f32, f32)>>
    <<UNKNOWN SSA VALUE>> : tensor<3924480x!llvm.struct<"struct.float3", (f32, f32, f32)>>
    <<UNKNOWN SSA VALUE>> : tensor<1x!llvm.struct<"struct.Config", (i8, i32, i32, i32, i32, f64, ptr, ptr, i32, i32, i8, i8, i8, i8, i8, i8, i8, ptr, ptr, ptr, ptr, ptr, ptr, i8, i8, i32, i32, i8, f64, i32, f64, i32, i32, i32, f64, f64, ptr, ptr, ptr, ptr, ptr, ptr, ptr, ptr, ptr, ptr, ptr, i32, i32, f64, f64, f64, f64, f64, ptr, ptr, ptr, i32, ptr)>>
  out
   